# UMLS Drug -[treats]- UMLS Disease
Starts with mesh-mesh, filters mesh-mesh with evidence for its mechanism, translates to mesh-umls

In [1]:
# Some of the source code: https://github.com/mmayers12/metapaths/blob/main/1_code/08_CTD.ipynb
import json
import pandas as pd
import csv
import numpy as np
import os
from biomedkg_utils import *

In [2]:
umls2mesh = json.load(open('output/otherMappings/umls2mesh_all.json'))
db2mesh = json.load(open('output/compound2compound/db2mesh.json'))
mesh2db = json.load(open('output/compound2compound/mesh2db.json'))

## Source: CTD

In [4]:
#os.remove('input/CTD_chemicals_diseases.csv.gz')
try: 
    os.remove('input/CTD_chemicals_diseases.csv')
except:
    pass
os.system('wget -N -P input/ http://ctdbase.org/reports/CTD_chemicals_diseases.csv.gz')
os.system('gunzip input/CTD_chemicals_diseases.csv.gz .')

--2023-03-22 22:33:54--  http://ctdbase.org/reports/CTD_chemicals_diseases.csv.gz
Resolving ctdbase.org (ctdbase.org)... 152.7.178.67
Connecting to ctdbase.org (ctdbase.org)|152.7.178.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138106677 (132M) [application/x-gzip]
Saving to: ‘input/CTD_chemicals_diseases.csv.gz’

     0K .......... .......... .......... .......... ..........  0%  270K 8m20s
    50K .......... .......... .......... .......... ..........  0%  526K 6m18s
   100K .......... .......... .......... .......... ..........  0%  503K 5m41s
   150K .......... .......... .......... .......... ..........  0%  164M 4m16s
   200K .......... .......... .......... .......... ..........  0%  498K 4m19s
   250K .......... .......... .......... .......... ..........  0%  338M 3m36s
   300K .......... .......... .......... .......... ..........  0%  495K 3m44s
   350K .......... .......... .......... .......... ..........  0%  238M 3m16s
   400K .......... .

  5200K .......... .......... .......... .......... ..........  3%  543K 3m3s
  5250K .......... .......... .......... .......... ..........  3%  520K 3m4s
  5300K .......... .......... .......... .......... ..........  3%  188M 3m2s
  5350K .......... .......... .......... .......... ..........  4%  515K 3m2s
  5400K .......... .......... .......... .......... ..........  4%  235M 3m1s
  5450K .......... .......... .......... .......... ..........  4%  516K 3m1s
  5500K .......... .......... .......... .......... ..........  4% 10.9M 3m0s
  5550K .......... .......... .......... .......... ..........  4%  545K 3m0s
  5600K .......... .......... .......... .......... ..........  4% 10.1M 2m59s
  5650K .......... .......... .......... .......... ..........  4%  543K 2m59s
  5700K .......... .......... .......... .......... ..........  4% 10.4M 2m57s
  5750K .......... .......... .......... .......... ..........  4%  541K 2m58s
  5800K .......... .......... .......... .......... ........

 10500K .......... .......... .......... .......... ..........  7%  532K 2m24s
 10550K .......... .......... .......... .......... ..........  7%  313M 2m24s
 10600K .......... .......... .......... .......... ..........  7% 8.68M 2m23s
 10650K .......... .......... .......... .......... ..........  7%  523K 2m23s
 10700K .......... .......... .......... .......... ..........  7%  362M 2m23s
 10750K .......... .......... .......... .......... ..........  8%  489K 2m23s
 10800K .......... .......... .......... .......... ..........  8%  306M 2m22s
 10850K .......... .......... .......... .......... ..........  8% 8.03M 2m22s
 10900K .......... .......... .......... .......... ..........  8%  526K 2m22s
 10950K .......... .......... .......... .......... ..........  8%  321M 2m21s
 11000K .......... .......... .......... .......... ..........  8% 7.23M 2m21s
 11050K .......... .......... .......... .......... ..........  8%  523K 2m21s
 11100K .......... .......... .......... .......... 

 15800K .......... .......... .......... .......... .......... 11% 7.41M 2m3s
 15850K .......... .......... .......... .......... .......... 11%  532K 2m4s
 15900K .......... .......... .......... .......... .......... 11% 7.48M 2m3s
 15950K .......... .......... .......... .......... .......... 11%  536K 2m4s
 16000K .......... .......... .......... .......... .......... 11%  350M 2m3s
 16050K .......... .......... .......... .......... .......... 11% 8.33M 2m3s
 16100K .......... .......... .......... .......... .......... 11%  530K 2m3s
 16150K .......... .......... .......... .......... .......... 12%  368M 2m3s
 16200K .......... .......... .......... .......... .......... 12%  486K 2m3s
 16250K .......... .......... .......... .......... .......... 12%  189M 2m3s
 16300K .......... .......... .......... .......... .......... 12% 6.66M 2m2s
 16350K .......... .......... .......... .......... .......... 12%  533K 2m2s
 16400K .......... .......... .......... .......... .......... 1

 21150K .......... .......... .......... .......... .......... 15% 2.55M 1m51s
 21200K .......... .......... .......... .......... .......... 15%  687K 1m51s
 21250K .......... .......... .......... .......... .......... 15% 2.65M 1m50s
 21300K .......... .......... .......... .......... .......... 15%  973K 1m50s
 21350K .......... .......... .......... .......... .......... 15% 1.93M 1m50s
 21400K .......... .......... .......... .......... .......... 15% 2.46M 1m50s
 21450K .......... .......... .......... .......... .......... 15%  720K 1m50s
 21500K .......... .......... .......... .......... .......... 15% 2.80M 1m50s
 21550K .......... .......... .......... .......... .......... 16%  672K 1m50s
 21600K .......... .......... .......... .......... .......... 16% 2.93M 1m50s
 21650K .......... .......... .......... .......... .......... 16% 1.05M 1m50s
 21700K .......... .......... .......... .......... .......... 16% 1.51M 1m50s
 21750K .......... .......... .......... .......... 

...

118300K .......... .......... .......... .......... .......... 87%  506K 15s
118350K .......... .......... .......... .......... .......... 87% 44.2M 15s
118400K .......... .......... .......... .......... .......... 87%  518K 15s
118450K .......... .......... .......... .......... .......... 87% 1.04M 15s
118500K .......... .......... .......... .......... .......... 87%  981K 15s
118550K .......... .......... .......... .......... .......... 87% 1.28M 15s
118600K .......... .......... .......... .......... .......... 87%  837K 15s
118650K .......... .......... .......... .......... .......... 88% 1.67M 15s
118700K .......... .......... .......... .......... .......... 88%  715K 15s
118750K .......... .......... .......... .......... .......... 88% 2.56M 15s
118800K .......... .......... .......... .......... .......... 88%  631K 15s
118850K .......... .......... .......... .......... .......... 88% 5.06M 15s
118900K .......... .......... .......... .......... .......... 88%  562K 15s

123800K .......... .......... .......... .......... .......... 91%  535K 10s
123850K .......... .......... .......... .......... .......... 91%  317M 10s
123900K .......... .......... .......... .......... .......... 91% 6.25M 10s
123950K .......... .......... .......... .......... .......... 91%  549K 10s
124000K .......... .......... .......... .......... .......... 91% 7.77M 10s
124050K .......... .......... .......... .......... .......... 92%  540K 10s
124100K .......... .......... .......... .......... .......... 92%  363M 10s
124150K .......... .......... .......... .......... .......... 92% 7.92M 10s
124200K .......... .......... .......... .......... .......... 92%  538K 10s
124250K .......... .......... .......... .......... .......... 92% 8.19M 10s
124300K .......... .......... .......... .......... .......... 92%  532K 10s
124350K .......... .......... .......... .......... .......... 92%  362M 10s
124400K .......... .......... .......... .......... .......... 92% 7.28M 9s


129350K .......... .......... .......... .......... .......... 95%  532K 5s
129400K .......... .......... .......... .......... .......... 95%  368M 5s
129450K .......... .......... .......... .......... .......... 96% 6.99M 5s
129500K .......... .......... .......... .......... .......... 96%  537K 5s
129550K .......... .......... .......... .......... .......... 96% 7.54M 5s
129600K .......... .......... .......... .......... .......... 96%  535K 5s
129650K .......... .......... .......... .......... .......... 96%  167M 5s
129700K .......... .......... .......... .......... .......... 96% 6.55M 5s
129750K .......... .......... .......... .......... .......... 96%  533K 5s
129800K .......... .......... .......... .......... .......... 96%  309M 5s
129850K .......... .......... .......... .......... .......... 96% 7.07M 4s
129900K .......... .......... .......... .......... .......... 96%  534K 4s
129950K .......... .......... .......... .......... .......... 96%  369M 4s
130000K ....

134850K .......... .........                                  100%  247K=2m5s

2023-03-22 22:36:00 (1.05 MB/s) - ‘input/CTD_chemicals_diseases.csv.gz’ saved [138106677/138106677]

gzip: . is a directory -- ignored


512

In [3]:
rows = [] 
with open('input/CTD_chemicals_diseases.csv','rt') as fin:
    data = csv.reader(fin)
    
    for i, row in enumerate(data):
        if i == 27:  #NOTE: This may change with new downloads
            headers = row
        else:
            rows.append(row)
            
df = pd.DataFrame(rows, columns=headers)
df.tail()

,# ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedIDs
8276734,zymosterol,C015582,128-33-6,"Bone Diseases, Metabolic",MESH:D001851,,CYP27A1,5.98,,24381012
8276735,zymosterol,C015582,128-33-6,Cholestasis,MESH:D002779,,CYP27A1,4.89,,15795599
8276736,zymosterol,C015582,128-33-6,Liver Cirrhosis,MESH:D008103,,CYP27A1,4.96,,24381012
8276737,zymosterol,C015582,128-33-6,"MICROCEPHALY, CONGENITAL CATARACT, AND PSORIAS...",OMIM:616834,,MSMO1,7.80,616834,
8276738,zymosterol,C015582,128-33-6,"Xanthomatosis, Cerebrotendinous",MESH:D019294,,CYP27A1,7.89,213700,12117727|15795599|17444890|19801147


In [4]:
df2 = df.DirectEvidence.replace(r'^\s*$', np.nan, regex=True)
df['DirectEvidence'] = df2
df['DirectEvidence'].value_counts()

cd_edges = df.dropna(subset=['DirectEvidence'])[['ChemicalID', 'DiseaseID', 'DirectEvidence']]
temp = cd_edges.DiseaseID.str.replace('MESH:', '')
cd_edges.loc[:,'DiseaseID'] = temp
grouped = cd_edges.groupby(['DirectEvidence'])
print(len(cd_edges), 'rows')
cd_edges.tail()

104626 rows


,ChemicalID,DiseaseID,DirectEvidence
8276057,D015054,D010538,marker/mechanism
8276406,D015054,D018805,marker/mechanism
8276429,D015054,D012769,marker/mechanism
8276675,D015054,D014605,marker/mechanism
8276731,C056855,D002806,marker/mechanism


In [9]:
therapeutic = grouped.get_group('therapeutic')
therapeutic.to_csv('output/compound2disease/chem2disease_treats.csv')
marker_or_mechanism = grouped.get_group('marker/mechanism')
marker_or_mechanism.to_csv('output/compound2disease/chem2disease_marker.csv')

db2mesh = json.load(open('output/compound2compound/db2mesh.json'))
meshDrugs = list(set([mesh[0] for mesh in list((db2mesh.values()))]))
tChem = list(set(therapeutic['ChemicalID']))
mChem = list(set(marker_or_mechanism['ChemicalID']))
t,m,tot  = 0,0,0
for chem in meshDrugs:
    if chem in tChem:
        t += 1
    if chem in mChem:
        m += 1
    if chem in mChem or chem in tChem:
        tot += 1
print(t,'/', len(tChem), 'Drugs with a drug-treats-disease relationship that have a with DrugBank2MeSH ID')
print(m,'/', len(mChem), 'Drugs with a drug-marker-disease relationship that have a with DrugBank2MeSH ID')
print(tot,'in either')
print(len(meshDrugs),'/',len(db2mesh), 'MeSH drugs of drugs with Drugbank-MeSH ID')

drugInd_ther = 0
for mesh in therapeutic['ChemicalID']:
    if mesh in meshDrugs:
        drugInd_ther += 1
print(drugInd_ther, 'drug-treats-disease relationships')

drugInd_mark = 0
for mesh in marker_or_mechanism['ChemicalID']:
    if mesh in meshDrugs:
        drugInd_mark += 1
print(drugInd_mark, 'drug-marker-disease relationships')

2710 / 6886 Drugs with a drug-treats-disease relationship that have a with DrugBank2MeSH ID
2487 / 6503 Drugs with a drug-marker-disease relationship that have a with DrugBank2MeSH ID
3329 in either
7781 / 8046 MeSH drugs of drugs with Drugbank-MeSH ID
26146 drug-treats-disease relationships
44610 drug-marker-disease relationships


In [10]:
# Export Drug-Disease 
compTREATSdis_ctd, compMARKERdis_ctd = dict(), dict()

for i in range(0,len(cd_edges)):
    comp = cd_edges['ChemicalID'].iloc[i]
    dis = cd_edges['DiseaseID'].iloc[i]
    rel = cd_edges['DirectEvidence'].iloc[i]

    if rel == 'therapeutic':
        compTREATSdis_ctd.setdefault(comp,[]).append(dis)
    elif rel == 'marker/mechanism':
        compMARKERdis_ctd.setdefault(comp,[]).append(dis)
            
with open('output/compound2disease/meshCompound-TREATS-meshDisease_ctd.json','w') as fout:
    json.dump(compTREATSdis_ctd, fout)
                             
with open('output/compound2disease/meshCompound-MARKER_OF-meshDisease_ctd.json','w') as fout:
    json.dump(compMARKERdis_ctd, fout)                            

### Source: PathFX (Guney + repoDB)
DrugBank - MeSH Disease from PathFX (Guney+repoDB)

In [11]:
# Note: All drugbank drugs here have at least one UMLS CUI for itself
pathfx = pd.read_csv('input/PathFX_DrugBank_to_UMLSdisease.csv')[['DrugBankID','Indication CUI']]
pathfx

,DrugBankID,Indication CUI
0,DB00783,C0001144
1,DB00741,C0001144
2,DB00936,C0001144
3,DB00799,C0001144
4,DB00759,C0001144
...,...,...
1751,DB00661,not mapped
1752,DB00570,not mapped
1753,DB00541,not mapped
1754,DB00162,not mapped


In [12]:
# Map MeSH drug to MeSH disease indication
compTREATSdis_pathfx = dict()
for i in range(0,len(pathfx)):
    
    try:
        # Drug/Compound
        db_drug = pathfx['DrugBankID'][i]
        mesh_comps = db2mesh[db_drug]
        
        # Disease
        umls_dis = pathfx['Indication CUI'][i]
        meshes_dis = umls2mesh[umls_dis]
        
        # Drug-treats-disease
        for mesh_dis in meshes_dis:
            for mesh_comp in mesh_comps:
                compTREATSdis_pathfx.setdefault(mesh_comp,set()).add(mesh_dis)
    
    except:
        continue
        

for k in compTREATSdis_pathfx.copy():
    compTREATSdis_pathfx[k] = list(compTREATSdis_pathfx[k])
        
with open('output/compound2disease/meshCompound-TREATS-meshDisease_pathfx.json','w') as fout:
    json.dump(compTREATSdis_pathfx, fout)

## Source: Hetionet

In [13]:
#!wget -N -P input/ http://sideeffects.embl.de/media/download/meddra_all_indications.tsv.gz
#!gunzip 'input/meddra_all_indications.tsv.gz'

##### NOTE: The older version of this file was used in Hetionet. 
##### I did not add the new terms to the hetionet edges

with open('input/meddra_all_indications.tsv') as fin:
    for i,line in enumerate(fin):
        line = line.split('\t')
        umls_id = line[1]
        name = line[3]
        concept_type = line[4]
        umls_meddra = line[5]
        name_meddra = line[6]
        if umls_id != umls_meddra:
            print(line)
        
#1: STITCH compound id (flat, see above)
#2: UMLS concept id as it was found on the label
#3: method of detection: NLP_indication / NLP_precondition / text_mention
#4: concept name
#5: MedDRA concept type (LLT = lowest level term, PT = preferred term; in a few cases the term is neither LLT nor PT)
#6: UMLS concept id for MedDRA term
#7: MedDRA concept name

#All side effects found on the labels are given as LLT. Additionally, the PT is shown. There is at least one
#PT for every LLT, but sometimes the PT is the same as the LLT.

['CID100000085', 'C0026827', 'text_mention', 'Muscle hypotonia', 'PT', 'C1142135', 'Muscle relaxant therapy\n']
['CID100000119', 'C0001768', 'text_mention', 'Agammaglobulinemia', 'PT', 'C0086438', 'Hypogammaglobulinaemia\n']
['CID100000119', 'C0085393', 'text_mention', 'Immunocompromised Host', 'PT', 'C0021051', 'Immunodeficiency\n']
['CID100000119', 'C0740380', 'text_mention', 'Varicella zoster', 'PT', 'C0019360', 'Herpes zoster\n']
['CID100000137', 'C0022602', 'NLP_indication', 'Actinic keratosis', 'PT', 'C0022603', 'Seborrhoeic keratosis\n']
['CID100000137', 'C0162568', 'NLP_precondition', 'Erythropoietic Protoporphyria', 'PT', 'C0853026', 'Porphyria non-acute\n']
['CID100000143', 'C0009402', 'NLP_indication', 'Colorectal Carcinoma', 'PT', 'C0007102', 'Colon cancer\n']
['CID100000143', 'C0009402', 'NLP_indication', 'Colorectal Carcinoma', 'PT', 'C1527249', 'Colorectal cancer\n']
['CID100000143', 'C0023895', 'text_mention', 'Liver diseases', 'PT', 'C0151763', 'Hepatocellular injury\n

['CID100003640', 'C0024299', 'NLP_indication', 'Lymphoma', 'PT', 'C0024305', "Non-Hodgkin's lymphoma\n"]
['CID100003640', 'C0033687', 'text_mention', 'Proteinuria', 'PT', 'C0948586', 'Protein urine present\n']
['CID100003640', 'C0033775', 'NLP_indication', 'Pruritus Ani', 'PT', 'C0033777', 'Pruritus genital\n']
['CID100003640', 'C0033775', 'NLP_indication', 'Pruritus Ani', 'PT', 'C2242737', 'Anorectal discomfort\n']
['CID100003640', 'C0035457', 'text_mention', 'Rhinitis, Allergic, Perennial', 'PT', 'C0848309', 'Rhinitis perennial\n']
['CID100003640', 'C0038325', 'text_mention', 'Stevens-Johnson Syndrome', 'PT', 'C0014742', 'Erythema multiforme\n']
['CID100003640', 'C0038814', 'NLP_indication', 'Sunburn', 'PT', 'C0162830', 'Photosensitivity reaction\n']
['CID100003640', 'C0041948', 'text_mention', 'Uremia', 'PT', 'C0242528', 'Azotaemia\n']
['CID100003640', 'C0042167', 'text_mention', 'Uveitis, Posterior', 'PT', 'C0008526', 'Choroiditis\n']
['CID100003640', 'C0043117', 'text_mention', 'P

['CID100012453', 'C0025362', 'text_mention', 'Mental Retardation', 'PT', 'C1306341', 'Mental disability\n']
['CID100012453', 'C0042693', 'text_mention', 'Violence', 'PT', 'C0001807', 'Aggression\n']
['CID100012453', 'C0221765', 'text_mention', 'Chronic schizophrenia', 'PT', 'C0036341', 'Schizophrenia\n']
['CID100012555', 'C0242429', 'NLP_indication', 'Sore Throat', 'PT', 'C2363731', 'Oropharyngeal pain\n']
['CID100012555', 'C0242429', 'NLP_indication', 'Sore Throat', 'PT', 'C2363730', 'Oropharyngeal discomfort\n']
['CID100012555', 'C1699324', 'text_mention', 'Pharyngeal mucositis', 'PT', 'C0031350', 'Pharyngitis\n']
['CID100012559', 'C0014121', 'NLP_indication', 'Bacterial Endocarditis', 'PT', 'C0014118', 'Endocarditis\n']
['CID100012559', 'C0023241', 'NLP_indication', "Legionnaires' Disease", 'PT', 'C0023240', 'Legionella infection\n']
['CID100012559', 'C0035436', 'NLP_indication', 'Rheumatic Fever', 'PT', 'C3495832', 'Rheumatic disorder\n']
['CID100012559', 'C1565489', 'NLP_precondit

### DOID Name -[is]-> DOID ID

In [24]:
hetionet_url = 'https://raw.githubusercontent.com/dhimmel/indications/gh-pages/data/indications-slim-collapsed.tsv'
!wget -N -P input/ $hetionet_url
db_name2id = dict()
doid_name2id = dict()
with open('input/db_name_doid_name.tsv') as fin:
    for line in fin:
        line = line.split('\t')
        db_id = line[0]
        db_name = line[1]
        db_name2id.setdefault(db_name,set()).add(db_id)
        
        doid_id = line[2]
        doid_name = line[3]
        doid_name2id.setdefault(doid_name,set()).add(doid_id)

--2023-03-15 23:05:34--  https://raw.githubusercontent.com/dhimmel/indications/gh-pages/data/indications-slim-collapsed.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71982 (70K) [text/plain]
Saving to: ‘input/indications-slim-collapsed.tsv’

indications-slim-co 100%[===================>]  70.29K  --.-KB/s    in 0.01s   

Last-modified header missing -- time-stamps turned off.
2023-03-15 23:05:34 (6.30 MB/s) - ‘input/indications-slim-collapsed.tsv’ saved [71982/71982]



## DOID ID -[is]- MeSH Disease ID

In [25]:
! wget -N -P input/ https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/main/src/ontology/HumanDO.obo

doid2mesh = dict()
with open('input/HumanDO.obo') as fin:
    IDs = []
    meshes = []
    for i,line in enumerate(fin):
        if i < 29:
            continue
        
        # New DOID term
        if '[Term]' in line:
            if len(IDs) > 0:
                meshes = set(meshes)
                for ID in IDs:
                    for mesh in meshes:
                        doid2mesh.setdefault(ID,set()).add(mesh)
                IDs = []
                meshes = []
                
        # DOID IDs
        elif 'id' in line.split(':')[0]:
            IDs.append(line.split(':')[1].strip()+':'+line.split(':')[2].strip())
        
        # Get UMLS CUIs
        if 'xref: UMLS_CUI:' in line:
            umls = line.split(':')[2].strip()
            try: meshes += umls2mesh[umls]
            except: continue
        
        # Get MeSH via UMLS CUIs
        if 'xref: MESH:' in line:
            mesh = line.split(':')[2].strip()
            meshes.append(mesh)

            
for k in doid2mesh.copy():
    doid2mesh[k] = list(doid2mesh[k])
        
with open('output/disease2disease/doid2mesh.json','w') as fout:
    json.dump(doid2mesh, fout)

--2023-03-15 23:05:35--  https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/main/src/ontology/HumanDO.obo
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6177093 (5.9M) [text/plain]
Saving to: ‘input/HumanDO.obo’

HumanDO.obo         100%[===================>]   5.89M  --.-KB/s    in 0.09s   

Last-modified header missing -- time-stamps turned off.
2023-03-15 23:05:35 (66.6 MB/s) - ‘input/HumanDO.obo’ saved [6177093/6177093]



## DrugBank Drug ID -[treats]-> MeSH Disease

In [14]:
!wget -N -P input/ https://raw.githubusercontent.com/dhimmel/indications/49cf8e3858f124e552051fc1e8c1c229a43dadf6/curation/results-three-curators.tsv
compTREATSdis_hetionet = dict()
with open('input/results-three-curators.tsv.txt') as fin:
    for line in fin:
        line = line.split('\t')
        try:
            diseases = doid2mesh[line[8]]
            comps = db2mesh[line[9]]
            moa = line[10]

            if 'DM' in moa: #if the drug treats the disease
                for comp in comps:
                    for disease in diseases:
                        compTREATSdis_hetionet.setdefault(comp,set()).add(disease)
        except:
            continue
            
# Compound-treats-Disease from Hetionet
count = 0
for k,v in compTREATSdis_hetionet.items():
    for V in v:
        count += 1
print(count, 'Compound-treats-disease from Hetionet')

--2023-03-22 22:37:25--  https://raw.githubusercontent.com/dhimmel/indications/49cf8e3858f124e552051fc1e8c1c229a43dadf6/curation/results-three-curators.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119212 (116K) [text/plain]
Saving to: ‘input/results-three-curators.tsv’

results-three-curat 100%[===================>] 116.42K  --.-KB/s    in 0.01s   

Last-modified header missing -- time-stamps turned off.
2023-03-22 22:37:26 (8.53 MB/s) - ‘input/results-three-curators.tsv’ saved [119212/119212]

0 Compound-treats-disease from Hetionet


# Merge Sources: CTD, PathFX, Hetionet

In [17]:
''' Merge compound-treats-disease dictionaries '''
def merge_dict_valuesAreLists(dMain, d2):
    for key,vals in d2.items():
        if key in dMain:
            new_vals = set(dMain[key])
            for val in vals:
                new_vals.add(val)
            dMain[key] = list(new_vals)
        else:
            dMain[key] = vals

compTREATSdis = compTREATSdis_ctd.copy()
merge_dict_valuesAreLists(compTREATSdis, compTREATSdis_pathfx)
merge_dict_valuesAreLists(compTREATSdis, compTREATSdis_hetionet)

''' Print summary counts '''
rels = 0
for k,v in compTREATSdis.items():
    for V in v:
        rels += 1
print(rels, 'compound-treats-disease relationships')

38368 compound-treats-disease relationships


In [19]:
# MeSH Compound -TREATS-> MeSH Disease
output_edgefile_onerel_noweight(outpath = 'output/compound2disease/edges_meshCompound-TREATS->meshDisease.csv',
                               columns = ['Compound (MeSH)','Disease (MeSH)','Relationship'],
                               dictionary = compTREATSdis,
                               rel = '-treats->',
                               prefix_col1 = 'MeSH_Compound:',
                               prefix_col2 = 'MeSH_Disease:')

df = pd.read_csv('output/edges/edges_meshCompound-TREATS->meshDisease.csv')
df.to_csv('output/edges to use/Compound_(MeSH)_treats_Disease_(MeSH).csv', index=False)

In [20]:
dbcompTREATSdis = dict()

for comp, diseases in compTREATSdis.items():
    try:
        db_comps = mesh2db[comp]
        for db in db_comps:
            for disease in diseases:
                dbcompTREATSdis.setdefault(db, list()).append(disease)
    except:
        continue

In [21]:
# drugbank Compound -TREATS-> drugbank Disease
output_edgefile_onerel_noweight(outpath = 'output/compound2disease/edges_drugbankCompound-TREATS->meshDisease.csv',
                               columns = ['Compound (DrugBank)','Disease (MeSH)','Relationship'],
                               dictionary = dbcompTREATSdis,
                               rel = '-treats->',
                               prefix_col1 = 'DrugBank_Compound:',
                               prefix_col2 = 'MeSH_Disease:')

df = pd.read_csv('output/edges/edges_drugbankCompound-TREATS->meshDisease.csv')
df.to_csv('output/edges to use/Compound_(DrugBank)_treats_Disease_(MeSH).csv', index=False)